In [1]:
# 필요한 파이썬 라이브러리를 설치합니다.
# 아래의 명령어를 통해 FastAPI와 기타 필요한 도구들을 설치할 수 있습니다.
!pip install fastapi uvicorn oracledb pandas nest_asyncio scikit-learn

In [2]:
# FastAPI: 웹 서버를 쉽게 만들어주는 도구입니다.
from fastapi import FastAPI

# oracledb: 오라클 데이터베이스와 연결하여 데이터를 가져오거나 저장하는 도구입니다.
import oracledb

# pandas(pd): 데이터를 엑셀처럼 표 형태로 쉽게 다룰 수 있게 해주는 도구입니다.
import pandas as pd

#nest_asyncio: 비동기(asyncio) 작업을 주피터 노트북에서 쉽게 사용할 수 있게 해주는 도구입니다.
import nest_asyncio

# uvicorn: FastAPI 서버를 실행하기 위한 도구입니다.
import uvicorn

# pickle: 파이썬 데이터를 파일 형태로 저장하거나 다시 읽을 때 사용하는 도구입니다.
import pickle

# cosine_similarity: 두 데이터가 얼마나 비슷한지 계산해주는 도구입니다.
from sklearn.metrics.pairwise import cosine_similarity

# BaseModel: 데이터를 정확한 형태로 주고받기 위해 사용하는 도구입니다.
from pydantic import BaseModel

# numpy(np): 여러 숫자 데이터를 쉽게 계산하고 처리할 수 있도록 도와주는 도구입니다.
import numpy as np

# CORSMiddleware: 보안 때문에 막혀 있는 데이터를 다른 서버나 사이트에서 접근할 수 있도록 허용해주는 도구입니다.
from fastapi.middleware.cors import CORSMiddleware

In [3]:
# FastAPI 웹 애플리케이션을 만듭니다.
app = FastAPI()

In [4]:
# 다른 사이트에서도 이 FastAPI 서버를 사용할 수 있도록 허용 설정을 추가합니다.
app.add_middleware(
    CORSMiddleware,
    allow_origins = ["*"],       # 모든 사이트에서의 접근을 허용합니다.
    allow_credentials = True,    # 자격증명(로그인 등)을 허용합니다.
    allow_methods = ["*"],       # 모든 방식(GET, POST 등)의 요청을 허용합니다.
    allow_headers = ["*"]        # 모든 헤더(부가정보)를 허용합니다.
)

In [5]:
# 웹 브라우저엥서 "/movie/hello_backend" 주소로 접근하면 작동하는 기능을 만듭니다.
@app.get("/movie/hello_backend")
async def hello():
    return "안녕 FastAPI"

In [6]:
# 오라클 데이터베이스와 연결하기 위한 설정을 만듭니다.
# oracledb.makedsn() 함수는 데이터베이스 연결 주소를 만드는 기능을 합니다.
# 함수의 매개변수는 다음과 같습니다:
# "localhost": 데이터베이스가 설치된 컴퓨터의 주소입니다. 여기서는 같은 컴퓨터(내 컴퓨터)를 의미합니다.
# 1521: 데이터베이스가 사용하는 포트 번호입니다. 보통 오라클은 1521 포트를 사용합니다.
# service_name="XEPDB1": 데이터베이스 안의 서비스 이름을 의미합니다. 오라클 설치 시 기본 서비스 이름입니다.
userdsn = oracledb.makedsn("localhost", 1521, service_name="XEPDB1")

# username 변수는 데이터베이스에 접속할 때 사뇽하는 사용자 이름을 저장합니다.
# "movie_scott"은 실제 데이터베이스에 존재하는 사용자 이름이어야 합니다.
username = "movie_scott"

# userpass 변수는 데이터베이스에 접속할 때 사용하는 사용자 비밀번호를 저장합니다.
# "pass123"은 실제 데이터베이스 사용자의 비밀번호이어야 합니다.
userpass = "pass123"

In [7]:
# @app.get()은 FastAPI에서 특정 주소로 요청이 들어오면 동작할 기능(함수)을 정의하는 부분입니다.
# 여기서는 사용자가 웹 브라우저에서 "/movie/movie_popular" 주소로 접속하면 실행됩니다.
@app.get("/movie/movie_popular")
async def movie_popular():
    # 오라클 데이터베이서와 연결하기 위해 oracledb.connect()를 사용합니다.
    # connect() 함수의 매개변수는 다음과 같습니다:
    # user: 데이터베이스 사용자 이름을 입력합니다.
    # password: 데이터베이스 사용자의 비밀번호를 입력합니다.
    # dsn: 데이터베이스의 위치나 주소를 입력합니다.
    connection = oracledb.connect(user=username, password=userpass, dsn=userdsn)

    # 데이터베이스에서 가져올 최신 영화 정보를 조회하기 위한 SQL 퀴리(명령어)를 작성합니다.
    # SELECT는 데이터를 가져올 때 사용하는 명령어입니다.
    # num, title, director 등은 데이터베이스(movie_tbl)에 있는 영화 정보를 나타내는 열(컬럼) 이름들입니다.
    # ORDER BY box_office DESC는 관객 수(box_office)가 가장 많은 순서대로 정렬하라는 의미입니다.
    #FETCH FIRST 30 ROWS ONLY는 위에서부터 30개의 데이터만 가져오라는 의미입니다.
    query = """
        SELECT num, title, director
              ,actor, box_office
              ,synopsis, poster
              ,open_date, degree
              ,country, movie_time
          FROM movie_tbl
         ORDER BY box_office DESC
         FETCH FIRST 30 ROWS ONLY
    """

    # 작성된 쿼리를 실행하여 결과를 pandas라는 라이브러리의 데이터프레임(엑셀처럼 표 형식) 형태로 저장합니다.
    # pd.read_sql() 함수의 매개변수는 다음과 같습니다:
    # query: 위에서 작성한 SQL 명령어입니다.
    # con: 데이터베이스와 연결된 정보를 가진 변수를 입력합니다.
    movie_df = pd.read_sql(query, con=connection)

    # 데이터베이스와 연결된 정보를 가진 변수를 입력합니다.
    connection.close()

    # movie_df 데이터를 Python 딕셔너리(사전) 형태로 변환하여 반환합니다.
    # movie_df.to_dict() 함수는 데이터를 표 형태에서 딕셔너리 형태로 바꿔줍니다.
    # orient='records'는 표의 각 행(가로줄)을 하나의 딕셔너리로 변환하라는 의미입니다.
    # 예를 들어 한 영화읭 정보가 {"title": "파묘", "director": "장재현"}과 같은 형태로 저장됩니다.
    return movie_df.to_dict(orient='records')

In [8]:
# 사용자의 요청에서 받을 데이터 형태를 정의하는 클래스(틀)를 만듭니다.
# MovieRequest는 사용자 요청 데이터가 어떻게 생겼는지 정의하는 역할을 합니다.
# BaseModel을 상속받아서 데이터를 쉽게 관리할 수 있게 도와줍니다.
class MovieRequest(BaseModel):
    num: int   # num은 사용자가 선택한 영화의 번호를 나타냅니다. 숫자 형태입니다.

In [9]:
# POST 방식으로 "/movie/movie_recommend" 주소에 요청을 하면 동작하는 기능을 만듭니다.
@app.post("/movie/movie_recommend")
async def movie_recommend(request: MovieRequest):

    # 데이터베이스에 연결하여 데이터를 가져오기 위한 연결 설정을 합니다.
    connection = oracledb.connect(user=username, password=userpass, dsn=userdsn)

    # movie_tbl이라는 테이블에서 영화 데이터를 가져오는 SQL 쿼리를 작성합니다.
    query = "SELECT * FROM movie_tbl"
    # 작성된 쿼리를 실행해서 결과를 표 형태(movie_df)의 데이터로 변환하여 저장합니다.
    movie_df = pd.read_sql(query, con=connection)

    # 데이터베이스에서 불러온 데이터를 다시 사용 가능한 형태로 변환하는 함수입니다.
    # pickle.loads() 함수는 압축ㄱ된 데이터를 원래 형태로 되돌려줍니다.
    def load_vector(blob):
        return pickle.loads(blob.read())

    # movie_df에서 SYNOPSIS_VECTOR 열에 있는 데이터를 다시 사용 가능한 형태로 변환합니다.
    movie_df["SYNOPSIS_VECTOR"] = movie_df["SYNOPSIS_VECTOR"].apply(load_vector)
    
    # 데이터베이스 연결을 종료합니다.
    connection.close()

    # 사용자가 요청한 영ㅎ와 번호(request.num)와 일치하는 영화 데이터를 movie_df에서 찾습니다.
    target_movie = movie_df[movie_df["NUM"] == request.num]

    # 만약 사용자가 요청한 영화가 없다면 빈 결과를 반환합니다.
    if target_movie.empty:
        return {}

    # 사용자가 선택한 영화의 줄거리 벡터를 가져옵니다.
    target_vector = target_movie.iloc[0]["SYNOPSIS_VECTOR"]

    # 모든 영화의 줄거리 벡터를 하나의 배열롤 합칩니다.
    vectors = np.vstack(movie_df["SYNOPSIS_VECTOR"].values)

    # cosine_similarity 함수를 사용해 선택한 영화와 다른 영화들의 유사도를 계산합니다.
    similarity_scores = cosine_similarity([target_vector], vectors)[0]

    #계산된 유사도 점수를 movie_df에 새로운 열(similarity)로 추가합니다.
    movie_df["similarity"] = similarity_scores

    # 사용자가 선택한 영화와 다른 영화 중 유사도가 높은 상위 20개 영화를 추천 영화로 선정합니다.
    recommended_movies = movie_df[movie_df["NUM"] != request.num].sort_values(
        by ="similarity", ascending=False).head(20)

    # 추천 영화들의 번호를 리스트 형태로 저장합니다.
    recommeded_num = recommended_movies["NUM"].tolist()

    # 추천 영화 번호를 이용해 movie_df에서 해당 영화정보를 선택합니다.
    recommed_df = movie_df[movie_df["NUM"].isin(recommeded_num)]
    # 추천 영화 정보에서 SYNOPSIS_VECTOR 열은 제거하영 사용자에게 보여줍니다.
    recommed_df = recommed_df.drop('SYNOPSIS_VECTOR', axis=1)

    # 최종적으로 추천 영화 정보를 딕셔너리 형태로 변환하여 반환합니다.
    return recommed_df.to_dict(orient='records')

In [10]:
# nest_asyncio라는 도구를 사용하여 비동기(async) 작업을 처리합니다.
# apply() 함수는 현재 사용하는 환경(특히 주피터 노트북 환경)에서 비동기 작업이 충돌하지 않고 정상적으로 작동하도록 설정하는 역할을 합니다.
nest_asyncio.apply()

In [ ]:
# uvicorn이라는 도구를 사용하여 위에서 만든 FastAPI 앱(app)을 실제로 웹 서버로 실행합니다.
# uvicorn.run() 함수는 서버를 실행할 때 사용합니다.
# 아래 매개변수는 서버 실행시 필요한 옵션입니다.
# app: 실행할 FastAPI 애플리케잉션을 지정합니다.
# host="0.0.0.0": 서버의 주소를 의미합니다. "0.0.0.0"은 모든 컴퓨터에서 이 서버에 접근 가능하도록 설정합니다.
# port=7070: 서버가 사용할 포트 번호를 의미합니다. 포트 번호는 특정 서비스를 찾을 때 사용하는 번호입니다. 여기서는 7070번 포트를 사용합니다.
uvicorn.run(app, host="0.0.0.0", port=7070)

INFO:     Started server process [20184]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7070 (Press CTRL+C to quit)


INFO:     127.0.0.1:62404 - "POST /movie/movie_recommend HTTP/1.1" 200 OK


C:\Users\User\AppData\Local\Temp\ipykernel_20184\3959481606.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  movie_df = pd.read_sql(query, con=connection)
